# Transform 2020: GemPy Workshop

## Table of Content

- Why is GemPy
- First Surface
- Enplaning data structures 1: Surfaces, Surface Points, Orientations
- Exercise: Finishing the model

## Why GemPy?

## Installation

1. Open conda prompt (Windows) or terminal (Linux/MacOS)
2. conda create --clone tutorials --name t20-gempy
3. https://docs.gempy.org/installation.html

## Your first model

In [2]:
# Importing GemPy
import gempy as gp

# Importing aux libraries
from ipywidgets import interact
import numpy as np
import matplotlib.image as mpimg

# Embedding matplotlib figures in the notebooks
%matplotlib qt5

### Initializing the model... But first a word about Grids:

The first step to create a GemPy model is create a `gempy.Model` object that will
contain all the other data structures and necessary functionality. In addition
 for this example we will define a *regular grid* since the beginning.
This is the grid where we will interpolate the 3D geological model.

GemPy is based on a **meshless intepolator**. In practice this means that we can
interpolate any point in a 3D space. However, for convenience, we have built some
standard grids for different purposes. At the current day the standard grids are:

- **Regular grid**: default grid mainly for general visualization
- **Custom grid**: GemPy's wrapper to interpolate on a user grid
- **Topography**: Topographic data use to be of high density. Treating it as an independent
  grid allow for high resolution geological maps
- **Sections**: If we predefine the section 2D grid we can directly interpolate at those
  locations for perfect, high resolution estimations
- **Center grids**: Half sphere grids around a given point at surface. This are specially tuned
  for geophysical forward computations

For now we will use a 2.5D regular grid

In [2]:
geo_model = gp.create_model('Transform-2020')
geo_model = gp.init_data(geo_model,
                         extent= [0, 791, 0, 200, -582, 0], # of the regular grid
                         resolution=[100, 10, 100])         # of the regular grid

Active grids: ['regular']


GemPy core code is written in Python. However for efficiency and gradient based
machine learning most of heavy computations happen in optimize compile code,
 either C or CUDA for GPU.

To do so, GemPy rely on the library `Theano`. To guarantee maximum optimization
`Theano` requires to compile the code for every Python kernel. The compilation is
done by calling the following line at any point (before computing the model):

In [3]:
gp.set_interpolator(
    geo_model,
    output=['geology'], # In built outputs. More about this later on
    theano_optimizer='fast_compile') # alternatives: 'fast_run' or
                                     # check http://deeplearning.net/software/theano/tutorial/modes.html                        

Setting kriging parameters to their default values.
Compiling theano function...
Level of Optimization:  fast_compile
Device:  cpu
Precision:  float64
Number of faults:  0
Compilation Done!
Kriging values: 
                   values
range             1002.2
$C_o$            23914.4
drift equations      [3]


### Creating figure:

GemPy uses `matplotlib` and `pyvista` for 2d and 3d visualization of the model respectively. One of the
design decisions of GemPy is to allow real time construction of the model. What this means is that you can start
adding input data and see in real time how the 3D surfaces evolve. Lets initialize the visualization windows.

The first one is the 2d figure. Just place the window where you can see it (maybe move the jupyter notebook to half
screen and use the other half for the renderers).

In [4]:
p2d = gp.plot_2d(geo_model, section_names=None,
                 direction=None, cell_number=None)

#### Add model section

In the 2d renderer we can add several cross section of the model. In this case, for simplicity sake we are just
adding one perpendicular to y.

In [5]:
# In this case perpendicular to the y axes
ax = p2d.add_section(cell_number=1, direction='y')

#### Loading cross-section image:

Remember that `gempy` is simply using `matplotlib` and therefore the ax object
 created above is a standard `matplotlib` axes.

Lets load an image with the information of couple of boreholes

In [6]:
# Reading image
img = mpimg.imread('wells.png')
# Plotting it inplace
ax.imshow(img, origin='upper', alpha=.8, extent = (0, 791, -582,0))

We can do the same in 3D through `pyvista` and vtk rendering.

Click the qt5 button Back (+Y) to have the same view as in the 2D viewer:

In [7]:
p3d = gp.plot_3d(geo_model, plotter_type='background', notebook=False)

## Building the model

Now that we have the model initialize and the 2D and 3D render in place we can
 start the construction of the geological model. 

### Surfaces

GemPy is a surface based interpolator. This means that all the input data we add has to be refereed to a surface. The
 surfaces always mark the **bottom** of a unit. 
 
 By default GemPy surfaces are empty:

In [8]:
geo_model.surfaces

,surface,series,order_surfaces,color,id


If we do not care about the names and we just want to interpolate a surface we can use.
It is important to notice that **the bottom
most layer will be considered always basement and therefore not interpolated**. Still
we can add values or change the color of that "surface" (for lack of a better name) to
populate the lower most volume.


In [13]:
# Default surfaces:
geo_model.set_default_surfaces()

,surface,series,order_surfaces,color,id
0,surface1,Default series,1,#015482,1
1,surface2,Default series,2,#9f0052,2


In [13]:
geo_model.surfaces


,surface,series,order_surfaces,color,id


We will see more about data structures below.

Now we can start adding data. GemPy input data consist on surface points and
orientations (perpendicular to the layers). The 2D plot gives you the X and Z coordinates when hovering the mouse
 over. We can add a surface point as follows:


In [17]:
# Add a point. If idx is None, next available index will be used. Here we pass it
# explicitly for consistency
geo_model.add_surface_points(X=223, Y=0.01, Z=-94, surface='surface1', idx=0)

# Plot in 2D
p2d.plot_data(ax, cell_number=11)

# Plot in 3D
p3d.plot_data()

Now we can add the other two points of the layer:

In [16]:
# Add points
geo_model.add_surface_points(X=458, Y=0, Z=-107, surface='surface1', idx=1)
geo_model.add_surface_points(X=612, Y=0, Z=-14, surface='surface1', idx=2)

# Plotting
p2d.plot_data(ax, cell_number=11)
p3d.plot_surface_points()

(vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor)00000291BD279648

The **minimum amount of input data** to interpolate anything in `gempy` is:

a) 2 surface points per surface

b) One orientation per series.

Lets add an orientation anywhere in space:

In [15]:
# Adding orientation
geo_model.add_orientations(X=350, Y=0, Z=-300, surface='surface1', pole_vector= (0,0,1))
p2d.plot_data(ax, cell_number=5)
p3d.plot_data()

Now we have enough data to finally **interpolate**!

In [18]:
gp.compute_model(geo_model)


Lithology ids 
  [2. 2. 2. ... 2. 2. 2.] 

That is, we have interpolated the 3D surface. We can visualize with:

In [19]:
# In 2D
p2d.plot_contacts(ax, cell_number=5)

# In 3D
p3d.plot_surfaces()

{'surface1': (vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor)00000291BD289CA8}

Congratulations! You have learnt how to create a surface. From here on, we will
learn how to combine multiple surfaces to little by little construct a full realize
3D structural model.

-----------
## Intermission: GemPy data classes

GemPy depends on multiple data objects to store all the data structures necessary
to construct an structural model. To keep all the necessary objects in sync the
class `gempy.ImplicitCoKriging` (which `geo_model` is instance of) will provide the
necessary methods to update these data structures coherently.

At current state (gempy 2.2), the data classes are:

- `gempy.SurfacePoints`
- `gempy.Orientations`
- `gempy.Surfaces`
- `gempy.Stack` (combination of `gempy.Series` and `gempy.Faults`)
- `gempy.Grid`
- `gempy.AdditionalData`
- `gempy.Solutions`

Today we will look into details only some of these classes but what is important
to notice is that you can access these objects as follows:

In [ ]:
# The default surfaces once again
geo_model.surfaces

In [20]:
# Also the surface points
geo_model.surface_points

,X,Y,Z,smooth,surface
0,223.0,0.01,-94.0,0.000001,surface1
1,458.0,0.00,-107.0,0.000001,surface1
2,612.0,0.00,-14.0,0.000001,surface1


In [21]:
# To find the location of the surface we just plot
geo_model.solutions.vertices

[array([[  3.955     ,  10.        , -64.28490875],
        [  3.955     ,  30.        , -64.3000058 ],
        [ 11.865     ,  10.        , -64.6319632 ],
        ...,
        [624.85240288, 110.        ,  -2.91      ],
        [623.23234711, 130.        ,  -2.91      ],
        [621.12407188, 150.        ,  -2.91      ]])]

In [24]:
# The grid values:
geo_model.grid, len(geo_model.grid.values)

(Grid Object. Values: 
 array([[   3.955,   10.   , -579.09 ],
        [   3.955,   10.   , -573.27 ],
        [   3.955,   10.   , -567.45 ],
        ...,
        [ 787.045,  190.   ,  -14.55 ],
        [ 787.045,  190.   ,   -8.73 ],
        [ 787.045,  190.   ,   -2.91 ]]),
 100000)

In [26]:
# And its correspondent ids:
geo_model.solutions.lith_block, len(geo_model.solutions.lith_block)

(array([2., 2., 2., ..., 2., 2., 2.]), 100000)

------

## Adding more layers

<div class="alert alert-block alert-warning">
<b>Exercise:</b> So far we only need 2 units defined. The cross-section image that
 we load have 4 however. Lets add two layers more:
</div>

### Layer 2

First we need to add some more layers.

In [28]:
# We can add already two extra surfaces
geo_model.add_surfaces(['surface3', 'basement'])

ValueError: new categories must not include old categories: {'basement', 'surface3'}

In [29]:
# Your code here:
geo_model.add_surface_points(X=225, Y=0, Z=-269, surface='surface2', idx=3)
geo_model.add_surface_points(X=459, Y=0, Z=-279, surface='surface2', idx=4)

#--------------------
# Plot data
p2d.plot_data(ax)
p3d.plot_data()

In [30]:
# Compute model
gp.compute_model(geo_model)

# Plot 2D
p2d.plot_lith(ax, cell_number=5)
p2d.plot_contacts(ax, cell_number=5)

# Plot 3D
p3d.plot_surfaces()
p3d.plot_structured_grid(opacity=.2, annotations = {1: 'surface1', 2:'surface2', 3:'surface3'})


[StructuredGrid (0x291c66bedc8)
   N Cells:	88209
   N Points:	100000
   X Bounds:	3.955e+00, 7.870e+02
   Y Bounds:	1.000e+01, 1.900e+02
   Z Bounds:	-5.791e+02, -2.910e+00
   Dimensions:	100, 10, 100
   N Arrays:	2,

### Layer 3

In [31]:
# Your code here:
# Add points
geo_model.add_surface_points(X=225, Y=0, Z=-439, surface='surface3')
geo_model.add_surface_points(X=464, Y=0, Z=-456, surface='surface3')
geo_model.add_surface_points(X=619, Y=0, Z=-433, surface='surface3')

# Compute model
gp.compute_model(geo_model)

# Plotting
p2d.plot_data(ax, cell_number=5)
p2d.plot_lith(ax, cell_number=5)
p2d.plot_contacts(ax, cell_number=5)

p3d.plot_surfaces()
p3d.plot_structured_grid(opacity=.2, annotations = {1: 'surface1', 2:'surface2', 3:'surface3', 4:'basement'})
p3d.plot_data()
# ------------------

In [35]:
# gp.save_model_to_pickle(geo_model, 'checkpoint1')

True

-------------------
## Intermission: GemPy data structures: Surfaces Points, Orientations and surfaces

On GemPy each surface is not independent of each other. Some surfaces will be
subparallel to each other while other surfaces will define a fault plane where
an offset may occur.

In order to account for all possible combinations, we use **categories** (in many
instances literal `pandas.CategoricalDtype`) and **ordered** `pandas.Dataframes`.
Let's take a look at `gempy.SurfacePoints`:

In [33]:
geo_model.surface_points


,X,Y,Z,smooth,surface
1,458.0,0.00,-107.0,0.000001,surface1
2,612.0,0.00,-14.0,0.000001,surface1
0,223.0,0.01,-94.0,0.000001,surface1
3,225.0,0.00,-269.0,0.000001,surface2
4,459.0,0.00,-279.0,0.000001,surface2
5,225.0,0.00,-439.0,0.000001,surface3
6,464.0,0.00,-456.0,0.000001,surface3
7,619.0,0.00,-433.0,0.000001,surface3


<div class="alert alert-block alert-info"><b>Checkpoint:</b> Next cell will load the
model from memory to the necessary state to continue.</div>

In [9]:
geo_model = gp.load_model_pickle('checkpoint1.pickle')

# Default plotting setting
p2d = gp.plot_2d(geo_model)
# Reading image
img = mpimg.imread('wells.png')
# Plotting it inplace
p2d.axes[0].imshow(img, origin='upper', alpha=.8, extent = (0, 791, -582,0))
p3d = gp.plot_3d(geo_model, plotter_type='background')

---------
## Unconformities and Faults:

So far the model is simply a depositional unit. GemPy allows for unconformities and faults to build complex models. This input is given by categorical data. In general:

input data (surface points/ orientations) <belong to< surface <belong to< series

And series can be a fault---i.e. offset the rest of surface--- or not. We are going to show how to add a fault as an example.

First we need to add a series:

In [24]:
geo_model.add_series('Fault1')

,order_series,BottomRelation,isActive,isFault,isFinite
Default series,1,Erosion,True,False,False
Fault1,2,Erosion,False,False,False


In [25]:
geo_model.reorder_series(['Fault1', 'Default series'])

,order_series,BottomRelation,isActive,isFault,isFinite
Fault1,1,Erosion,False,False,False
Default series,2,Erosion,True,False,False


Then define that is a fault:

In [26]:
geo_model.set_is_fault('Fault1')

Fault colors changed. If you do not like this behavior, set change_color to False.


,order_series,BottomRelation,isActive,isFault,isFinite
Fault1,1,Fault,False,True,False
Default series,2,Erosion,True,False,False


But we also need to add a new surface:

In [27]:
geo_model.add_surfaces('fault1')

,surface,series,order_surfaces,color,id
0,surface1,Default series,1,#015482,1
1,surface2,Default series,2,#9f0052,2
2,surface3,Default series,3,#ffbe00,3
3,basement,Default series,4,#728f02,4
4,fault1,Default series,5,#443988,5


And finally assign the new surface to the new series/fault

In [28]:
gp.map_series_to_surfaces(geo_model, {'Fault1':'fault1'})

,surface,series,order_surfaces,color,id
4,fault1,Fault1,1,#443988,1
0,surface1,Default series,1,#015482,2
1,surface2,Default series,2,#9f0052,3
2,surface3,Default series,3,#ffbe00,4
3,basement,Default series,4,#728f02,5


Now we can just add input data as before (remember the minimum amount of input data to compute a model):

In [29]:
# Add input data of the fault
geo_model.add_surface_points(X=550, Y=0, Z=-30, surface='fault1')
geo_model.add_surface_points(X=650, Y=0, Z=-200, surface='fault1')
geo_model.add_orientations(X=600, Y=0, Z= -100, surface='fault1',
                           pole_vector=(.3,0,.3))

# Plotting Inpute data
p2d.remove(ax)
p2d.plot_data(ax, cell_number=5)
p3d.plot_data()

And now is computing as before:

In [30]:
# Compute
gp.compute_model(geo_model)

# Plot
p2d.plot_lith(ax, cell_number=5)
p2d.plot_contacts(ax, cell_number=5)
p3d.plot_surfaces()
p3d.plot_structured_grid(opacity=.2, annotations = {2: 'surface1', 3:'surface2',
                                                    4:'surface3', 5:'basement'})

[StructuredGrid (0x16e0ad27288)
   N Cells:	88209
   N Points:	100000
   X Bounds:	3.955e+00, 7.870e+02
   Y Bounds:	1.000e+01, 1.900e+02
   Z Bounds:	-5.791e+02, -2.910e+00
   Dimensions:	100, 10, 100
   N Arrays:	3,

As you can see now instead of having folding layers we have a sharp jump. Building on this you can pretty much any model you can imagine.

## Additional features:

Over the years we have built a bunch of assets integrate with `gempy`. Here we will show some of them:

### Topography

GemPy has a built-in capabilities to read and manipulate topographic data (through gdal). To show an example we can
just create a random topography:

In [31]:
## Adding random topography
geo_model.set_topography(source='random',fd=1.9, d_z=np.array([-150, 0]), resolution=np.array([200,200]))

Active grids: ['regular' 'topography']


Grid Object. Values: 
array([[   3.955     ,   10.        , -579.09      ],
       [   3.955     ,   10.        , -573.27      ],
       [   3.955     ,   10.        , -567.45      ],
       ...,
       [ 791.        ,  197.98994975, -135.35107881],
       [ 791.        ,  198.99497487, -135.00724647],
       [ 791.        ,  200.        , -134.90309829]])

The topography can we visualize in both renderers:

In [32]:
p2d.plot_topography(ax, cell_number=5)
p3d.plot_topography(scalars='topography')

(vtkRenderingOpenGL2Python.vtkOpenGLActor)0000016E0CB781C8

But also allows us to compute the geological map of an area:

In [33]:
gp.compute_model(geo_model)
p3d.plot_surfaces()
p3d.plot_topography()

(vtkRenderingOpenGL2Python.vtkOpenGLActor)0000016E6E7ACCA8

In [34]:
p3d.plot_structured_grid()

[UnstructuredGrid (0x16e0cb33a68)
   N Cells:	75372
   N Points:	86930
   X Bounds:	3.955e+00, 7.870e+02
   Y Bounds:	1.000e+01, 1.900e+02
   Z Bounds:	-5.791e+02, -1.455e+01
   N Arrays:	3,

### Gravity inversion

GemPy also allows for inversions (in production only gravity so far). We can see a small demo how this works.

The first thing to do is to assign densities to each of the units:

In [35]:
geo_model.add_surface_values([0, 2.6, 2.4, 3.2, 3.6], ['density'])

,surface,series,order_surfaces,color,id,density
4,fault1,Fault1,1,#443988,1,0.000000
0,surface1,Default series,1,#015482,2,2.600000
1,surface2,Default series,2,#9f0052,3,2.400000
2,surface3,Default series,3,#ffbe00,4,3.200000
3,basement,Default series,4,#728f02,5,3.600000


Also we can create a centered grid around a device for precision:

In [36]:
geo_model.set_centered_grid(centers= [[400, 0, 0]],  resolution = [10, 10, 100], radius=800)

Active grids: ['regular' 'topography' 'centered']
Active grids: ['regular' 'topography' 'centered']


Grid Object. Values: 
array([[    3.955     ,    10.        ,  -579.09      ],
       [    3.955     ,    10.        ,  -573.27      ],
       [    3.955     ,    10.        ,  -567.45      ],
       ...,
       [ 1200.        ,   800.        ,  -922.71624587],
       [ 1200.        ,   800.        ,  -964.3665184 ],
       [ 1200.        ,   800.        , -1008.        ]])

We need to modify the compile code:

In [37]:
gp.set_interpolator(geo_model, output=['gravity'], theano_optimizer='fast_run')

Setting kriging parameters to their default values.
Compiling theano function...
Level of Optimization:  fast_run
Device:  cpu
Precision:  float64
Number of faults:  1
Compilation Done!
Kriging values: 
                   values
range             1002.2
$C_o$            23914.4
drift equations   [3, 3]


But now additionally to the interpolation we also compute the forward gravity of the model (at the point XYZ = 400, 0, 0)

In [38]:
gp.compute_model(geo_model)
geo_model.solutions.fw_gravity

array([-80.36323225])

We can visualize it it better in the following figure. The aim of an inversion is to find the set of parameters that 
fit a measured point the better. In this example the red x symbolize the measured gravity while  the blue dots are 
the current gravity (and previous) fw gravity values.  The widget moves up and down the surface 3

In [39]:
# Creating new figure
p_grav = gp.plot_2d(geo_model, direction=None, show=False, figsize=(6, 9))

# Adding section for model
ax2 = p_grav.add_section(cell_number=1, direction='y', ax_pos=211)

# Adding section for gravity 
ax3 = p_grav.add_section(ax_pos=414)

# Plotting model in section
p2d.plot_data(ax2, cell_number=5, legend='force')
p2d.plot_lith(ax2, cell_number=5)
p2d.plot_contacts(ax2, cell_number=5)
ax2.plot(400, 0, '^', markersize=40, c='red')

# Plotting initial values of the gravity axes
target_grav = -810
ax3.tick_params( bottom=False)
ax3.spines['top'].set_visible(False)
ax3.spines['bottom'].set_visible(False)
ax3.spines['right'].set_visible(False)
ax3.plot(0, target_grav, 'X', label='Target Gravity', markersize=4, c='red')
ax3.plot(1, geo_model.solutions.fw_gravity, 'o', label='Current Gravity', markersize=4, c='blue')
ax3.set_ylabel('grav')

# We store the original values of z for the surface 3
Z_ori = geo_model.surface_points.df.loc[[5,6,7], 'Z'].copy()
# init a list to store grav
grav_ = []

# Function that modify the model, compute it and plot
def gravity_invert(dz):
    new_z = Z_ori + dz
    geo_model.modify_surface_points(indices=[5, 6, 7], Z= new_z)
    gp.compute_model(geo_model)
    grav_.append(geo_model.solutions.fw_gravity[0])
    
    p2d.remove(ax2)
    p2d.plot_data(ax2, cell_number=5)
    p2d.plot_lith(ax2, cell_number=5)
    p2d.plot_contacts(ax2, cell_number=5)
    ax3.plot(np.arange(len(grav_))+1, grav_, 'o', label='Current Gravity', markersize=4, c='blue')
    ax3.set_xlim(-1, len(grav_)+1)
    
interact(gravity_invert, dz=(-200, 200, 10))

interactive(children=(IntSlider(value=0, description='dz', max=200, min=-200, step=10), Output()), _dom_classe…

<function __main__.gravity_invert(dz)>

In [46]:
# Creating new figure
p_grav = gp.plot_2d(geo_model, direction=None, show=False, figsize=(6, 9))

# Adding section for model
ax2 = p_grav.add_section(cell_number=1, direction='y', ax_pos=211)

# Adding section for gravity 
ax3 = p_grav.add_section(ax_pos=414)

# Plotting model in section
p2d.plot_data(ax2, cell_number=5, legend='force')
p2d.plot_lith(ax2, cell_number=5)
p2d.plot_contacts(ax2, cell_number=5)
ax2.plot(400, 0, '^', markersize=40, c='red')

# Plotting initial values of the gravity axes
target_grav = -81
ax3.tick_params( bottom=False)
ax3.spines['top'].set_visible(False)
ax3.spines['bottom'].set_visible(False)
ax3.spines['right'].set_visible(False)
ax3.plot(0, target_grav, 'X', label='Target Gravity', markersize=4, c='red')
ax3.plot(1, geo_model.solutions.fw_gravity, 'o', label='Current Gravity', markersize=4, c='blue')
ax3.set_ylabel('grav')

# We store the original values of z for the surface 3
Z_ori = geo_model.surface_points.df.loc[[5,6,7], 'Z'].copy()
# init a list to store grav
grav_ = []

# Function that modify the model, compute it and plot
def gravity_invert(dz):
    new_z = Z_ori + dz
    geo_model.modify_surface_points(indices=[5, 6, 7], Z= new_z)
    gp.compute_model(geo_model)
    grav_.append(geo_model.solutions.fw_gravity[0])
    
    p2d.remove(ax2)
    p2d.plot_data(ax2, cell_number=5)
    p2d.plot_lith(ax2, cell_number=5)
    p2d.plot_contacts(ax2, cell_number=5)
    ax3.plot((np.arange(len(grav_))+1), grav_, 'o', label='Current Gravity', markersize=4, c='blue')
    
interact(gravity_invert, dz=(-200, 200, 10))

interactive(children=(IntSlider(value=0, description='dz', max=200, min=-200, step=10), Output()), _dom_classe…

<function __main__.gravity_invert(dz)>